In [1]:
from interface import ObjectInterface
import numpy as np
import utils

FILENAME_DATABASE = 'data/database.db'

def get_interface() -> ObjectInterface:
    interface = ObjectInterface(FILENAME_DATABASE)
    interface.connect()
    return interface

In [6]:
with get_interface() as interface:
    obj = interface.get_object(2)
    print(obj.raw_attributes)
    print(obj['zip'])

    obj.modify(zip='PLZ 46539')
    print(obj.raw_attributes)

{'street': 'Möllerweg', 'house_number': '359c', 'zip': '12214', 'city': 'Fuchsingen'}
PLZ 12214
{'street': 'Möllerweg', 'house_number': '359c', 'zip': '46539', 'city': 'Fuchsingen'}


In [7]:
with get_interface() as interface:
    obj = interface.create_object(interface.get_class(name='ArrayData'), some_numbers=np.arange(10).reshape((2, -1)))
    obj_db = interface.get_object(obj.id)
    print(obj_db.get_raw_value('some_numbers'), len(obj_db.get_raw_value('some_numbers')), len(utils.array_to_bytes(obj_db.get_value('some_numbers'))))
    print(obj_db.get_unprocessed_value('some_numbers'))
    print(obj_db.get_value('some_numbers'))

b'x\x9c\x9b\xec\x17\xea\x1b\x10\xc9\xc8P\xc6P\xad\x9e\x92Z\x9c\\\xa4n\xa5\xa0n\x93i\xa2\xae\xa3\xa0\x9e\x96_TR\x94\x98\x17\x9f_\x94\x92\n\x12wK\xcc)N\x05\x8a\x17g$\x16\xa4\x02\xf9\x1aF:\n\xa6\x9a:\n\xb5\nd\x03.\x06 `\x04b& f\x06b\x16 f\x05b6 f\x07b\x0e \xe6\x04b\x00\xa4\xa7\x1b\x9e' 101 168
[[0 1 2 3 4]
 [5 6 7 8 9]]
[[0 1 2 3 4]
 [5 6 7 8 9]]
